# Differences in MT and SP in parasha #4: Vayera (Genesis 18:1-22:24)

## Table of Content <a class="anchor" id="TOC"></a> (ToC)

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Compare surface texts of SP and MT</a>
* <a href="#bullet4">4 - Compare texts using minimum Levenshtein distance</a>
* <a href="#bullet5">5 - Comparison of spelling of proper nouns between SP and MT</a>
* <a href="#bullet6">6 - References and acknowledgement</a>
* <a href="#bullet7">7 - Required libraries</a>
* <a href="#bullet8">8 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

The Samaritan Pentateuch (SP) is a version of the Torah preserved by the Samaritan community, differing from the Masoretic Text (MT) in several aspects, including language, orthography, and occasionally theological emphasis. This notebook compares the text of the Masoretic Text, based on the BHSA dataset in Text-Fabric, with the Samaritan Pentateuch, also available as a Text-Fabric dataset.<a href="#ref1"><sup>1</sup></a>

In this analysis, we focus on comparing the text of the verses in a specific parasha, highlighting differences in wording and orthography. Additionally, special attention is given to spelling variations of proper nouns between the two traditions. This notebook draws inspiration from the notebook provided by Martijn Naaijer<a href="#ref2"><sup>2</sup></a> and aims to explore the textual nuances between these two important versions of the Torah.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The following code will load the Text-Fabric version of the [Samaritan Pentatuch](https://github.com/DT-UCPH/sp), the [Biblia Hebraica Stuttgartensia (Amstelodamensis)](https://etcbc.github.io/bhsa/) together with the additonal parasha related features from [tonyjurg/BHSaddons](https://github.com/tonyjurg/BHSaddons).

In [1]:
from tf.app import use

# Load the SP data, and rename the node features class F,
# the locality class L and the text class T, 
# then they cannot be overwritten while loading the MT.
SP = use('DT-UCPH/sp', version='3.4')
Fsp, Lsp, Tsp = SP.api.F, SP.api.L, SP.api.T

# Do the same for the MT dataset (BHSA) together with BHSaddons 
MT = use('etcbc/bhsa', version='2021',mod="tonyjurg/BHSaddons/tf/")
Fmt, Lmt, Tmt = MT.api.F, MT.api.L, MT.api.T

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


# 3 - Compare surface texts of SP and MT <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

In this section, we compare the surface texts of the Samaritan Pentateuch (SP) and the Masoretic Text (MT) at the verse level. By analyzing the wording and structure of these texts, we aim to identify variations.

In [2]:
# find all word nodes for this parasha (we can either use the transliterated name or the sequence number)
parashaQuery = '''
verse parashanum=4
'''
parashaResults = MT.search(parashaQuery)

  0.01s 147 results


In [3]:
# Extract book, chapter, and verse information
bookChapterVerseList = [
    Tmt.sectionFromNode(verse[0]) for verse in parashaResults
]

# Store parashname, start and end verse for future use
startNode=parashaResults[0][0]
endNode=parashaResults[-1][0]
parashaNameHebrew=Fmt.parashahebr.v(startNode)
parashaNameEnglish=Fmt.parashatrans.v(startNode)
bookStart,chapterStart,startVerse=Tmt.sectionFromNode(startNode)
parashaStart=f'{bookStart} {chapterStart}:{startVerse}'
bookEnd,chapterEnd,startEnd=Tmt.sectionFromNode(endNode)
parashaEnd=f'{chapterEnd}:{startEnd}'

In [4]:
# Function to reconstruct verses
def reconstructVerses(F, L, T, textFeature, inputList):
    """Reconstruct text for each verse."""
    verseTexts = {}
    for verseName in inputList:
        verseText = ''
        verseNode = T.nodeFromSection(verseName)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            wordText = eval(f'F.{textFeature}.v(wordNode)')
            trailer = F.trailer.v(wordNode)
            if wordText:
                verseText += wordText + (trailer if trailer else ' ')
        verseTexts[verseName] = verseText.strip()
    return verseTexts
    
SPverses = reconstructVerses(Fsp, Lsp, Tsp, 'g_cons', bookChapterVerseList)
MTverses = reconstructVerses(Fmt, Lmt, Tmt, 'g_cons', bookChapterVerseList)

In [5]:
from difflib import SequenceMatcher

def highlightMatches(baseText, comparisonText):
    matcher = SequenceMatcher(None, baseText, comparisonText)
    highlightedComparisonText = "" 
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":  # Identical parts
            highlightedComparisonText += comparisonText[j1:j2]
        else:  # Non-matching parts
            highlightedComparisonText += f'<mark>{comparisonText[j1:j2]}</mark>'  
    return highlightedComparisonText

def cleanText(text):
    replacements = [
         # for the transcoded strings
         ('00_P', ''),  # Remove '00_P'
         ('00_S', ''),  # Remove '00_S'
         ('00', ''),    # Remove '00'
         ('&', ' '),    # Replace '&' with a space
         # for the Hebrew strings
         ('ס ', ''),    # Final Samekh
         ('פ ', ''),    # Final Pe
         ('׃', ''),     # End of verse
         ('־',' ')      # maqaf
    ]
    # Apply each replacement
    for old, new in replacements:
        text = text.replace(old, new)
    return text

# Function to format and highlight verse differences between MT and SP
def formatAndHighlight(label, MTverseText, SPverseText):
    book, chapter, verse = label
    MTverseNode = Tmt.nodeFromSection(label)
    MTtext = cleanText(Tmt.text(MTverseNode, "text-orig-plain"))
    SPverseNode = Tsp.nodeFromSection(label)
    SPtext = Tsp.text(SPverseNode)
    SPmarkedText = highlightMatches(MTtext, SPtext)
    MTmarkedText = highlightMatches(SPtext, MTtext)
    formattedDiff = (
        f'<h4>{book} {chapter}:{verse}</h4>'
        f'<p><b>SP:</b> {SPmarkedText}<br><b>MT:</b> {MTmarkedText}</p>'
    )
    return formattedDiff

# Create header
MT.dm(f'### Differences between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

# Highlight and display differences for SP and MT verses
for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    MTverseText=cleanText(MTverseText)
    if MTverseText != SPverseText:  # Check for differences
        difference = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(difference)

### Differences between MT and SP for parasha Vayera (Genesis 18:1-22:24)

<h4>Genesis 18:1</h4><p><b>SP:</b> וירא אליו יהוה באל<mark>ו</mark>ני ממרא והוא יש<mark></mark>ב פתח האהל כחם היום <br><b>MT:</b> וירא אליו יהוה באל<mark></mark>ני ממרא והוא יש<mark>ׁ</mark>ב פתח האהל כחם היום </p>

<h4>Genesis 18:2</h4><p><b>SP:</b> ויש<mark></mark>א עיניו וירא והנה ש<mark></mark>לש<mark></mark>ה אנ<mark>ו</mark>ש<mark></mark>ים נצבים עליו וירא וירץ לקראתם מפתח האהל ויש<mark></mark>תחו<mark>י</mark> ארצה <br><b>MT:</b> ויש<mark>ׂ</mark>א עיניו וירא והנה ש<mark>ׁ</mark>לש<mark>ׁ</mark>ה אנ<mark></mark>ש<mark>ׁ</mark>ים נצבים עליו וירא וירץ לקראתם מפתח האהל ויש<mark>ׁ</mark>תחו<mark></mark> ארצה </p>

<h4>Genesis 18:3</h4><p><b>SP:</b> ויאמר אדני אם נא מצאתי חן בעיני<mark>כם</mark> אל נא תעבר<mark>ו</mark> מעל עבד<mark>כם</mark> <br><b>MT:</b> ויאמר אדני אם נא מצאתי חן בעיני<mark>ך</mark> אל נא תעבר<mark></mark> מעל עבד<mark>ך</mark> </p>

<h4>Genesis 18:4</h4><p><b>SP:</b> יקח נא מעט מים ורחצו רגליכם והש<mark></mark>ע<mark>י</mark>נו תחת העץ <br><b>MT:</b> יקח נא מעט מים ורחצו רגליכם והש<mark>ׁ</mark>ע<mark></mark>נו תחת העץ </p>

...

<h4>Genesis 22:20</h4><p><b>SP:</b> ויהי אחר<mark></mark> הדברים האלה ויגד לאברהם לאמר הנה ילדה מלכה גם ה<mark>י</mark>א בנים לנחור אחיך <br><b>MT:</b> ויהי אחר<mark>י</mark> הדברים האלה ויגד לאברהם לאמר הנה ילדה מלכה גם ה<mark>ו</mark>א בנים לנחור אחיך </p>

<h4>Genesis 22:21</h4><p><b>SP:</b> את עוץ בכ<mark>ו</mark>רו ואת בוז אחיו ואת קמואל אבי ארם <br><b>MT:</b> את עוץ בכ<mark></mark>רו ואת בוז אחיו ואת קמואל אבי ארם </p>

<h4>Genesis 22:22</h4><p><b>SP:</b> ואת כש<mark></mark>ד ואת חזו ואת פ<mark>י</mark>לדש<mark></mark> ואת ידלף ואת בתואל <br><b>MT:</b> ואת כש<mark>ׂ</mark>ד ואת חזו ואת פ<mark></mark>לדש<mark>ׁ</mark> ואת ידלף ואת בתואל </p>

<h4>Genesis 22:23</h4><p><b>SP:</b> ובתואל <mark>הול</mark>י<mark></mark>ד את רבקה ש<mark></mark>מנה אלה ילדה מלכה לנחור אחי אברהם <br><b>MT:</b> ובתואל <mark>י</mark>ל<mark></mark>ד את רבקה ש<mark>ׁ</mark>מנה אלה ילדה מלכה לנחור אחי אברהם </p>

<h4>Genesis 22:24</h4><p><b>SP:</b> ופילגש<mark></mark>ו וש<mark></mark>מה ר<mark></mark>ומה ותלד גם ה<mark>י</mark>א את טבח ואת גחם ואת תחש<mark></mark> ואת מעכה <br><b>MT:</b> ופילגש<mark>ׁ</mark>ו וש<mark>ׁ</mark>מה ר<mark>א</mark>ומה ותלד גם ה<mark>ו</mark>א את טבח ואת גחם ואת תחש<mark>ׁ</mark> ואת מעכה </p>

# 4 - Compare texts using minimum Levenshtein distance<a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The Levenshtein distance measures the minimum number of single-character edits (insertions, deletions, or substitutions) needed to transform one text into another, providing a quantitative way to compare textual differences. For comparing the Masoretic Text and Samaritan Pentateuch, it highlights variations in spelling, word order, or minor textual changes. 
In the context of the Levenshtein distance (in the script below `threshold`), a higher number indicates greater dissimilarity between two texts, meaning more edits (insertions, deletions, or substitutions) are needed to transform one text into the other.

In [6]:
# Compare verses based on Levenshtein distance
from Levenshtein import distance
threshold = 20

# Create header
MT.dm(f'### Levenshtein distance {threshold} between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    if distance(MTverseText, SPverseText) > threshold:
        formattedDiff = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(formattedDiff)

### Levenshtein distance 20 between MT and SP for parasha Vayera (Genesis 18:1-22:24)

<h4>Genesis 20:13</h4><p><b>SP:</b> ויהי כאש<mark></mark>ר התע<mark>ה</mark> אתי אלהים מבית אבי ו<mark>מארץ מולדתי ו</mark>אמר לה זה חסד<mark>י</mark>ך אש<mark></mark>ר תעש<mark></mark>י<mark>ן</mark> עמדי אל כל המקום אש<mark></mark>ר נבוא ש<mark></mark>מה אמרי לי אחי הוא <br><b>MT:</b> ויהי כאש<mark>ׁ</mark>ר התע<mark>ו</mark> אתי אלהים מבית אבי ו<mark></mark>אמר לה זה חסד<mark></mark>ך אש<mark>ׁ</mark>ר תעש<mark>ׂ</mark>י<mark></mark> עמדי אל כל המקום אש<mark>ׁ</mark>ר נבוא ש<mark>ׁ</mark>מה אמרי לי אחי הוא </p>

# 5 - Comparison of spelling of proper nouns between SP and MT<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

This section focuses on comparing the spelling of proper nouns between the Samaritan Pentateuch (SP) and the Masoretic Text (MT). Proper nouns, including names of people, places, and unique terms, often exhibit variations in spelling

In [7]:
import collections

def collectProperNounSpellings(F, L, T, inputList):
    """
    Collect proper noun spellings and their associated word node numbers.
    Ensures only one tuple is stored for each lexeme-to-spelling mapping.
    """
    properNounsSpellings = {}
    for bookChapterVerse in inputList:
        verseNode = T.nodeFromSection(bookChapterVerse)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            if F.sp.v(wordNode) == 'nmpr':  # Check if the word is a proper noun
                lex = F.lex.v(wordNode)    # Lexical form
                spelling = F.g_cons.v(wordNode)  # Spelling
                # Store only the first occurrence for each lex-to-cons mapping
                if lex not in properNounsSpellings or spelling not in {item[0] for item in properNounsSpellings[lex]}:
                    properNounsSpellings.setdefault(lex, []).append((spelling, wordNode))
    return properNounsSpellings
        
SPspellingDict = collectProperNounSpellings(Fsp, Lsp, Tsp, bookChapterVerseList) 
MTspellingDict = collectProperNounSpellings(Fmt, Lmt, Tmt, bookChapterVerseList)

In [8]:
# Create header
MT.dm(f'### Spelling differences proper nouns in SP and MT for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')
    
for lex, MTspellings in MTspellingDict.items():
    # Retrieve SP spellings, defaulting to an empty set if lex is not found
    SPspellings = SPspellingDict.get(lex, set())

    # Extract only the spellings (ignoring node numbers) for comparison
    MTspellingSet = {spelling for spelling, _ in MTspellings}
    SPspellingSet = {spelling for spelling, _ in SPspellings}

    # Compare the sets of spellings
    if MTspellingSet != SPspellingSet:
        # Print MT spelling with reference
        MTnode = list(MTspellings)[0][1]  # Get first tuple's node number
        book, chapter, verse = Tmt.sectionFromNode(MTnode)
        MTgloss = Fmt.gloss.v(MTnode)
        MTspelling = Fmt.g_cons_utf8.v(MTnode)

        # Build Markdown output
        output = f"#### Word: **{MTgloss}** ({book} {chapter}:{verse})\n"
        output += f"- **MT Spelling:** `{MTspelling}`\n"

        # Print SP spellings with reference
        if SPspellings:
            SPnode = list(SPspellings)[0][1]  # Get first tuple's node number
            SPspelling = Fsp.g_cons_utf8.v(SPnode)
            output += f"- **SP Spelling:** `{SPspelling}`\n"
        else:
            output += "- **SP Spelling:** None\n"

        # Print the formatted output
        MT.dm(output)

### Spelling differences proper nouns in SP and MT for parasha Vayera (Genesis 18:1-22:24)

#### Word: **Zoar** (Genesis 19:22)
- **MT Spelling:** `צוער`
- **SP Spelling:** `צער`


#### Word: **Paran** (Genesis 21:21)
- **MT Spelling:** `פארן`
- **SP Spelling:** `פראן`


#### Word: **Phicol** (Genesis 21:22)
- **MT Spelling:** `פיכל`
- **SP Spelling:** None


#### Word: **Moriah** (Genesis 22:2)
- **MT Spelling:** `מריה`
- **SP Spelling:** None


#### Word: **Pildash** (Genesis 22:22)
- **MT Spelling:** `פלדשׁ`
- **SP Spelling:** `פילדשׁ`


#### Word: **Reumah** (Genesis 22:24)
- **MT Spelling:** `ראומה`
- **SP Spelling:** `רומה`


# 6 - References and acknowledgement <a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<a class="anchor" id="bullet1"><sup>1</sup></a> Christian Canu Højgaard, Martijn Naaijer, & Stefan Schorch. (2023). Text-Fabric Dataset of the Samaritan Pentateuch. Zenodo. https://doi.org/10.5281/zenodo.7734632

<a class="anchor" id="bullet2"><sup>2</sup> [Notebook created by Martijn Naaijer](https://github.com/DT-UCPH/sp/blob/main/notebooks/combine_sp_with_mt_data.ipynb)

# 7 - Required libraries <a class="anchor" id="bullet7"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    collections
    difflib
    Levenshtein

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.

# 8 - Notebook version details<a class="anchor" id="bullet8"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>17 November 2024</td>
    </tr>
  </table>
</div>